In [ ]:
!pip install rasterio
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 27.3 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
import numpy as np
import geemap
import ee
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import drive
import time

In [ ]:

# Autentication and Initialization
ee.Authenticate()
ee.Initialize(project="alessiodeluca")

drive.mount('/content/drive/', force_remount=True)
%cd /content/drive/My Drive/Environment/RemoteSensingProject/

%ls

Mounted at /content/drive/
/content/drive/My Drive/Environment/RemoteSensingProject
gee/  images/  Notebooks/


In [ ]:
# Definition of the Area of Interest (AOI) in Croatia
geometry = ee.Geometry.Point([16.563798, 43.496451])
roi = geometry.buffer(10000)

# Function to select the image based on the date
def get_image_by_date(collection, date):
    return collection.filterDate(date, date).first()

# Sentinel-2 image collection for July and August 2017
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate('2017-07-06', '2017-08-10')

# Extract and print the dates of the images in the collection
image_dates = collection.aggregate_array('system:time_start').getInfo()

# Convert the dates from timestamp to readable format
image_dates = [ee.Date(date).format('YYYY-MM-dd').getInfo() for date in image_dates]

# Print the dates
for i, date in enumerate(image_dates):
    print(f"Image {i + 1}: {date}")


Image 1: 2017-07-07
Image 2: 2017-07-17
Image 3: 2017-07-27
Image 4: 2017-08-06


In [ ]:

print(collection.size().getInfo())

4


In [ ]:
imageList = collection.toList(collection.size())

pre_fire_image = imageList.get(0)
pre_fire_image = ee.Image(pre_fire_image)

during_fire_image = imageList.get(1)
during_fire_image = ee.Image(during_fire_image)

post_fire_image = imageList.get(3)
post_fire_image = ee.Image(post_fire_image)

In [ ]:
# Select the RGB bands: B4 (Red), B3 (Green), B2 (Blue)
pre_fire_image = pre_fire_image.select(['B4', 'B3', 'B2'])
during_fire_image = during_fire_image.select(['B4', 'B3', 'B2'])
post_fire_image = post_fire_image.select(['B4', 'B3', 'B2'])


In [ ]:
# Function to export the image to Google Drive
def export_image_to_drive(image, description, folder, scale, region):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=region,
        maxPixels=1e13  # Allows exporting large images
    )
    task.start()

# Export images to Google Drive
export_image_to_drive(pre_fire_image, 'PreFireImageRGB', 'gee', 10, roi)
export_image_to_drive(during_fire_image, 'DuringFireImageRGB', 'gee', 10, roi)
export_image_to_drive(post_fire_image, 'PostFireImageRGB', 'gee', 10, roi)

print("Image export started.")


Image export started.


In [ ]:


# Function to monitor the task status
def monitor_task(task):
    while task.active():
        print(f'Task {task.status().get("description")} is {task.status().get("state")}')
        time.sleep(10)  # Wait 10 seconds before checking again
    print(f'Task {task.status().get("description")} completed with status: {task.status().get("state")}')

# Modify the export_image_to_drive function to return the task
def export_image_to_drive(image, description, folder, scale, region):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=region,
        maxPixels=1e13
    )
    task.start()
    return task  # Return the task for monitoring

# Execute the export and monitor the task status
pre_fire_task = export_image_to_drive(pre_fire_image, 'PreFireImageRGB', 'gee', 10, roi)
monitor_task(pre_fire_task)

during_fire_task = export_image_to_drive(during_fire_image, 'DuringFireImageRGB', 'gee', 10, roi)
monitor_task(during_fire_task)

post_fire_task = export_image_to_drive(post_fire_image, 'PostFireImageRGB', 'gee', 10, roi)
monitor_task(post_fire_task)


Task PreFireImageRGB is READY
Task PreFireImageRGB is READY
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB is RUNNING
Task PreFireImageRGB completed with status: COMPLETED
Task DuringFireImageRGB is READY
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB is RUNNING
Task DuringFireImageRGB completed with status: COMPLETED
Task PostFireImageRGB is READY
Task PostFireImageRGB is RUNNING
Task PostFireImageRGB is RUNNING
Task PostFireImageRGB is RUNNING
Task PostFireImageRGB is RUNNING
Task PostFireImageRGB is RUNNING
Task PostFireImageRGB is RUNNING
Task PostFireImageRGB is RUNNIN

In [ ]:
# Open the .tif image to check the number of bands
with rasterio.open('/content/drive/My Drive/Environment/RemoteSensingProject/gee/PreFireImageRGB.tif') as src:
    print(f"The image has {src.count} band(s)")


The image has 3 band(s)


In [ ]:


def scaleMinMax(x):
    """Normalizes the array to values between 0 and 1 using Min-Max scaling."""
    return (x - np.nanmin(x)) / (np.nanmax(x) - np.nanmin(x))

def scaleCCC(x):
    """Normalizes the array using contrast correction based on percentiles."""
    return (x - np.nanpercentile(x, 2)) / (np.nanpercentile(x, 98) - np.nanpercentile(x, 2))

def scaleStd(x):
    """Normalizes the array using standard deviation."""
    mean = np.nanmean(x)
    std = np.nanstd(x)
    return (x - (mean - 2 * std)) / ((mean + 2 * std) - (mean - 2 * std))

def read_and_normalize_image(file_path):
    """Reads and normalizes an RGB image from a GeoTIFF file."""
    with rasterio.open(file_path) as src:
        # Read the RGB bands (assuming bands are in the order R, G, B)
        red = src.read(1)
        green = src.read(2)
        blue = src.read(3)

        # Normalization using different techniques
        def normalize_band(band, method):
            """Normalizes a band using the specified method."""
            if method == 'MinMax':
                return scaleMinMax(band)
            elif method == 'CCC':
                return scaleCCC(band)
            elif method == 'Std':
                return scaleStd(band)
            else:
                raise ValueError("Unrecognized normalization method.")

        # Apply normalizations
        red_minmax = normalize_band(red, 'MinMax')
        green_minmax = normalize_band(green, 'MinMax')
        blue_minmax = normalize_band(blue, 'MinMax')

        rgb_minmax = np.dstack((red_minmax, green_minmax, blue_minmax))

        red_ccc = normalize_band(red, 'CCC')
        green_ccc = normalize_band(green, 'CCC')
        blue_ccc = normalize_band(blue, 'CCC')

        rgb_ccc = np.dstack((red_ccc, green_ccc, blue_ccc))

        red_std = normalize_band(red, 'Std')
        green_std = normalize_band(green, 'Std')
        blue_std = normalize_band(blue, 'Std')

        rgb_std = np.dstack((red_std, green_std, blue_std))

        return rgb_minmax, rgb_ccc, rgb_std

def display_images(file_paths):
    """Reads, normalizes, and displays RGB images from a list of GeoTIFF files."""
    num_files = len(file_paths)
    fig, axs = plt.subplots(num_files, 3, figsize=(18, num_files * 6))

    for i, file_path in enumerate(file_paths):
        rgb_minmax, rgb_ccc, rgb_std = read_and_normalize_image(file_path)

        axs[i, 0].imshow(rgb_minmax)
        axs[i, 0].axis('off')

        axs[i, 1].imshow(rgb_ccc)
        axs[i, 1].axis('off')

        axs[i, 2].imshow(rgb_std)
        axs[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

# Paths to GeoTIFF files
pre_fire_path = '/content/drive/My Drive/Environment/RemoteSensingProject/gee/PreFireImageRGB.tif'
during_fire_path = '/content/drive/My Drive/Environment/RemoteSensingProject/gee/DuringFireImageRGB.tif'
post_fire_path = '/content/drive/My Drive/Environment/RemoteSensingProject/gee/PostFireImageRGB.tif'

# Display the normalized images
display_images([pre_fire_path, during_fire_path, post_fire_path])


Output hidden; open in https://colab.research.google.com to view.